In [0]:
from pyspark.sql.functions import *

In [0]:
%sql
use catalog emrcatalog

In [0]:
df_hosa=spark.read.parquet('/mnt/bronze/hosa/patients')
df_hosa.createOrReplaceTempView("patients_hosa")

df_hosb=spark.read.parquet('/mnt/bronze/hosb/patients')
df_hosb.createOrReplaceTempView("patients_hosb")

In [0]:
%sql
select * from patients_hosa limit 10;

In [0]:
%sql

-- CDM (Common Data Model)

create or replace temp view cdm_patients as
select concat(src_patient_id,'-',datasource) as patient_key,* 
from 
(
  select PatientID as src_patient_id,
  FirstName,
  LastName,
  MiddleName,
  SSN,
  PhoneNumber,
  Gender,
  DOB,
  Address,
  ModifiedDate,
  datasource
  from patients_hosa
  union all
  select ID as src_patient_id,
  F_Name as FirstName,
  L_Name as LastName,
  M_Name as MiddleName,
  SSN,
  PhoneNumber,
  Gender,
  DOB,
  Address,
  Updated_Date as ModifiedDate,
  datasource
  from patients_hosb
)

In [0]:
%sql
select * from cdm_patients limit 10;

In [0]:
%sql
--quailty checks
create or replace temp view quality_checks as
select patient_key,src_patient_id,firstname,lastname,middlename,ssn,phonenumber,gender,dob,address,modifieddate,datasource,
case when src_patient_id is null or dob is null or firstname is null or lower(firstname)='null' then true else false end as is_quarantined
from cdm_patients

In [0]:
%sql
select is_quarantined,count(*) as cnt from quality_checks group by is_quarantined;

In [0]:
%sql
-- SCD2 -> Slowly Changing Dimension Type 2, we will monitor this using inserted_date,modified_date,is_current columns
create table if not exists silver.patients
(
  patient_key string,
  src_patient_id string,
  firstname string,
  lastname string,
  middlename string,
  ssn string,
  phonenumber string,
  gender string,
  dob date,
  address string,
  src_modifieddate timestamp,
  datasource string,
  is_quarantined boolean,
  inserted_date timestamp,
  modified_date timestamp,
  is_current boolean
)
using delta;

In [0]:
from pyspark.sql.functions import current_timestamp

In [0]:
%sql
-- step1 : mark existing records as historical (is_current=false) for patients will be updated
merge into silver.patients as target
using quality_checks as source
on target.patient_key=source.patient_key and target.is_current=true
when matched 
and -- check if any of below mentioned column is changing
(
  target.src_modifieddate!=source.modifieddate or
  target.src_patient_id!=source.src_patient_id or
  target.firstname!=source.firstname or
  target.lastname!=source.lastname or
  target.middlename!=source.middlename or
  target.ssn!=source.ssn or
  target.phonenumber!=source.phonenumber or
  target.gender!=source.gender or
  target.address!=source.address or
  target.datasource!=source.datasource or
  target.is_quarantined!=source.is_quarantined
)
then update set target.is_current=false,target.modified_date=current_timestamp() -- end the record 

when not matched -- we are seeing this record for the first time
then insert
(
  patient_key,
  src_patient_id,
  firstname,
  lastname,
  middlename,
  ssn,
  phonenumber,
  gender,
  dob,
  address,
  src_modifieddate,
  datasource,
  is_quarantined,
  inserted_date,
  modified_date,
  is_current
)
values
(
  source.patient_key,
  source.src_patient_id,
  source.firstname,
  source.lastname,
  source.middlename,
  source.ssn,
  source.phonenumber,
  source.gender,
  source.dob,
  source.address,
  source.modifieddate,
  source.datasource,
  source.is_quarantined,
  current_timestamp(), -- set current timestamp as inserted date
  current_timestamp(),
  true -- mark this record as active
);


In [0]:
%sql
-- incase when record is changing we ended the previous record, now we need to insert the new updated record

merge into silver.patients as target
using quality_checks as source
on target.patient_key=source.patient_key 
and target.is_current=true -- last we changed is_current to false
when not matched 
then insert
(
  patient_key,
  src_patient_id,
  firstname,
  lastname,
  middlename,
  ssn,
  phonenumber,
  gender,
  dob,
  address,
  src_modifieddate,
  datasource,
  is_quarantined,
  inserted_date,
  modified_date,
  is_current
)
values
(
  source.patient_key,
  source.src_patient_id,
  source.firstname,
  source.lastname,
  source.middlename,
  source.ssn,
  source.phonenumber,
  source.gender,
  source.dob,
  source.address,
  source.modifieddate,
  source.datasource,
  source.is_quarantined,
  current_timestamp(), -- set current timestamp as inserted date
  current_timestamp(),
  true -- mark this record as active
);


In [0]:
%sql
select patient_key,count(*) from silver.patients group by patient_key;